## Question 1

In [1]:
!mlflow --version

mlflow, version 1.26.1


## Question 2

Download the file

In [2]:
!wget -O "./data/green_tripdata_2021-01.parquet" "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet"
!wget -O "./data/green_tripdata_2021-02.parquet" "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet"
!wget -O "./data/green_tripdata_2021-03.parquet" "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet"

--2022-05-30 18:45:26--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.200.72
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.200.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2021-01.parquet’

./data/green_tripda 100%[===================>]   1.27M   516KB/s    in 2.5s    

2022-05-30 18:45:30 (516 KB/s) - ‘./data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-30 18:45:31--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.174
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2021-02.parquet’

./data/green_tripda 100%[================

In [5]:
!mkdir ./output
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

mkdir: cannot create directory ‘./output’: File exists


In [6]:
!ls ./output

dv.pkl	test.pkl  train.pkl  valid.pkl


## Question 3

First run test

In [7]:
!python train.py

Second run (with `mlflow.start_run()` and `mlflow.autolog()`) 

In [11]:
!python train.py

2022/05/30 19:05:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [13]:
ls ./mlruns/0/*/params

bootstrap     max_leaf_nodes         min_weight_fraction_leaf  verbose
ccp_alpha     max_samples            n_estimators              warm_start
criterion     min_impurity_decrease  n_jobs
max_depth     min_samples_leaf       oob_score
max_features  min_samples_split      random_state


## Question 4

In [17]:
!mlflow server --backend-store-uri sqlite:///mydb.sqlite --default-artifact-root ./artifacts

[2022-05-30 19:11:13 +0700] [27707] [INFO] Starting gunicorn 20.1.0
[2022-05-30 19:11:13 +0700] [27707] [INFO] Listening at: http://127.0.0.1:5000 (27707)
[2022-05-30 19:11:13 +0700] [27707] [INFO] Using worker: sync
[2022-05-30 19:11:13 +0700] [27712] [INFO] Booting worker with pid: 27712
[2022-05-30 19:11:13 +0700] [27713] [INFO] Booting worker with pid: 27713
[2022-05-30 19:11:13 +0700] [27714] [INFO] Booting worker with pid: 27714
[2022-05-30 19:11:13 +0700] [27715] [INFO] Booting worker with pid: 27715
^C
[2022-05-30 19:11:56 +0700] [27707] [INFO] Handling signal: int
[2022-05-30 19:11:56 +0700] [27713] [INFO] Worker exiting (pid: 27713)
[2022-05-30 19:11:56 +0700] [27715] [INFO] Worker exiting (pid: 27715)
[2022-05-30 19:11:56 +0700] [27714] [INFO] Worker exiting (pid: 27714)
[2022-05-30 19:11:56 +0700] [27712] [INFO] Worker exiting (pid: 27712)


## Question 5

In [19]:
!pip install hyperopt

     |████████████████████████████████| 1.6 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 7.0 MB/s eta 0:00:01
  Using cached future-0.18.2-py3-none-any.whl
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
You should consider upgrading via the '/home/desenfirman/development/mlops-zoomcamp/_homework/02-experiment-tracking/venv/bin/python -m pip install --upgrade pip' command.


In [28]:
!python hpo.py

2022/05/30 19:34:06 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|█████████| 50/50 [11:41<00:00, 14.04s/trial, best loss: 6.6284257482044735]


## Question 6

First run test of `register_model.py`

In [29]:
!python register_model.py

2022/05/30 19:59:09 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.


After modify the `register_model.py` code (by search lowest `test_rmse` value and register the model)

In [31]:
!python register_model.py

Successfully registered model 'random-forest-best-model'.
2022/05/30 20:21:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best-model, version 1
Created version '1' of model 'random-forest-best-model'.


Download experiment result into csv and view it

In [38]:
import pandas as pd
test_res = pd.read_csv("./random-forest-best-models-experiment-result.csv").sort_values(by="test_rmse", ascending=True).head()
test_res

,Start Time,Duration,Run ID,Name,Source Type,Source Name,User,Status,bootstrap,ccp_alpha,...,warm_start,test_rmse,training_mae,training_mse,training_r2_score,training_rmse,training_score,valid_rmse,estimator_name,estimator_class
0,2022-05-30 20:19:00,43.0s,bc9c6e32cc374f078b002608af29d0ec,NaN,LOCAL,register_model.py,desenfirman,FINISHED,True,0.0,...,False,6.547886,3.790040,31.131051,0.767166,5.579521,0.767166,6.628426,RandomForestRegressor,sklearn.ensemble._forest.RandomForestRegressor
1,2022-05-30 20:11:24,39.9s,1a76a5c2a5b0493db3d729ec0cc580de,NaN,LOCAL,register_model.py,desenfirman,FINISHED,True,0.0,...,False,6.547886,3.790040,31.131051,0.767166,5.579521,0.767166,6.628426,RandomForestRegressor,sklearn.ensemble._forest.RandomForestRegressor
2,2022-05-30 19:59:09,33.0s,9540867b542e475585ecf7360a8222df,NaN,LOCAL,register_model.py,desenfirman,FINISHED,True,0.0,...,False,6.547886,3.790040,31.131051,0.767166,5.579521,0.767166,6.628426,RandomForestRegressor,sklearn.ensemble._forest.RandomForestRegressor
3,2022-05-30 20:21:15,29.4s,38aca71cc97046f492c3dfba22f97ed3,NaN,LOCAL,register_model.py,desenfirman,FINISHED,True,0.0,...,False,6.548853,3.777622,30.585266,0.771248,5.530395,0.771248,6.629893,RandomForestRegressor,sklearn.ensemble._forest.RandomForestRegressor
4,2022-05-30 20:14:49,46.8s,587058eb00f945e7a917a85bacfeeaba,NaN,LOCAL,register_model.py,desenfirman,FINISHED,True,0.0,...,False,6.548853,3.777622,30.585266,0.771248,5.530395,0.771248,6.629893,RandomForestRegressor,sklearn.ensemble._forest.RandomForestRegressor


In [39]:
print(f"Best model: {test_res.test_rmse.min()}")

Best model: 6.547886185595423
